## 한글텍스트 - 분류 모델링
### 로지스틱 회귀 모델 with TF-IDF

### 라이브러리 불러오기

In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
# 경고메세지 무시
import warnings
warnings.filterwarnings(action='ignore') 

### 데이타 불러오기 - 텍스트 형태

In [3]:
DATA_IN_PATH = './data_in/' 
DATA_OUT_PATH = './data_out/'
TRAIN_CLEAN_DATA = 'nsmc_train_clean.csv'
TEST_CLEAN_DATA = 'nsmc_test_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [4]:
train_df = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA )
test_df = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA )

In [5]:
train_df.isna().sum().sum()

1289

In [6]:
# 결측치 제거
train_df = train_df.dropna()
test_df = test_df.dropna()

In [7]:
train_df.isna().sum().sum()

0

In [8]:
test_df.shape

(49562, 3)

### TF-IDF 벡터화
- min_df : 설정한 값보다 특정 토큰의 df 값이 적게 나오면 벡터화 과정에서 제거한다는 의미
- max_df : 설정한 값보다 특정 토큰의 df 값이 크게 나오면 벡터화 과정에서 제거한다는 의미
- ngram_range : 빈도의 기본 단위를 어느 범위의 n-gram으로 설정할 것인지를 지정하는 인자

#### 형태소 분석기 함수화

In [9]:
# from konlpy.tag import Okt

# okt = Okt()
# def okt_tokenizer(text):
#     # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
#     tokens_ko = okt.morphs(text)
#     return tokens_ko

# Mac 전용 Mecab 사용
from konlpy.tag import Mecab   

mecab = Mecab()
def mecab_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = mecab.morphs(text)
    return tokens_ko

In [10]:
# okt_tokenizer("꼭 대박나세요")

# Mac 전용 Mecab 사용
mecab_tokenizer("꼭 대박나세요")

['꼭', '대박', '나세', '요']

#### TF-IDF 피처벡터화

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Okt 객체의 morphs( ) 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2) 
# tfidf_vect = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)

# Mac 전용 Mecab 사용
tfidf_vect = TfidfVectorizer(tokenizer=mecab_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)

tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

In [12]:
# 단어 행렬 확인(DTM)
tfidf_matrix_train

<148711x108644 sparse matrix of type '<class 'numpy.float64'>'
	with 3111333 stored elements in Compressed Sparse Row format>

In [13]:
print(tfidf_vect.vocabulary_)

': 38261, '쿨 지': 99968, '여성 라면': 69764, '감독 기억': 1934, '니 그럴': 22891, '고 얕': 7431, '둘 봤': 31313, '재미있 구요': 86979, '밀려오 네요': 44948, '으로 가': 77394, '인 억지': 83096, '정돈 아니': 89308, '게 죽였': 5373, '게 더러운': 4862, '돼냐': 30784, '부리 는데': 49430, '로 는가': 35432, '살아남 았': 52805, '앳킨스': 63697, '개밥 맛': 3319, '싫 습니다': 59297, '에 라': 67382, '같 성격': 2917, '요코': 75374, '지만 아닌': 94260, '그리고 결혼': 12285, '더러웠 다': 28071, '을 와': 79315, '사이코 패스': 52466, '내용 별거': 18656, '삶 순간': 52873, '참 와': 96851, '마리화나': 37780, '재밌 뮤지컬': 87096, '장난치 면': 86421, '딴따라 더': 32712, '독단': 30185, '독단 적': 30186, '스토리 삼류': 57171, '종목': 91489, '만감 교차': 38931, '쿼': 99986, '다 내생': 23662, '적 관심': 87699, '를 정확히': 37180, '등가교환': 32267, '극장판 샴발라': 12663, '연출력 문제': 71048, '끄 영화': 15454, '결과 만': 5775, '케이블 용': 99638, '흐느끼': 108106, '누웠': 21434, '나올까요': 16895, '작품 차라리': 85716, '에겐 최악': 68578, '점수 짜': 89060, '영화 오프닝': 72409, '내 하루': 18324, '엠 블': 69497, '블 런': 50491, '리뷰 안': 37467, '일 뒤': 83937, '엔 제대로': 69382, '존재 며': 91055, '걸 세상': 4435, '만점 이런':

In [14]:
sentence = [train_df['document'][0]]
print(tfidf_vect.transform(sentence).toarray())

[[0. 0. 0. ... 0. 0. 0.]]


### 로지스틱 회귀 모델 분류

In [15]:
# Logistic Regression 을 이용하여 감성 분석 Classification 수행. 
lg_clf = LogisticRegression(random_state=0)

# 모델 학습 - model.fit(x_train, y_train)
lg_clf.fit(tfidf_matrix_train , train_df['label'] )

LogisticRegression(random_state=0)

### 테스트 데이타를 이용한 정확도 평가

In [16]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함. 
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# 테스트 데이타 예측
preds = lg_clf.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ',accuracy_score(test_df['label'], preds))

Logistic Regression 정확도:  0.8644122513215771


In [17]:
# 예측값
pred = lg_clf.predict(tfidf_matrix_test)

In [18]:
# 실제값
test_df['label']

0        1
2        0
3        0
4        0
5        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: label, Length: 49562, dtype: int64

In [19]:
# 정확도
sum(pred == test_df['label'] )

42842

In [20]:
42749/49564

0.8625010087967073

### 로지스틱 회귀 모델 분류 & GridSearchCV

In [25]:
from sklearn.model_selection import GridSearchCV

# Logistic Regression 을 이용하여 감성 분석 Classification 수행. 
lg_clf = LogisticRegression(random_state=0)

# Parameter C 최적화를 위해 GridSearchCV 를 이용. 
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lg_clf , param_grid=params , cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train , train_df['label'] )
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 3.5} 0.8645


### 테스트 데이타에 TF-IDF 벡터 변환 후 정확도 평가

In [26]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함. 
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier 는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ',accuracy_score(test_df['label'],preds))

Logistic Regression 정확도:  0.8679431822767443


# 로지스틱 회귀모델 with Word2Vec
# 랜덤포레스트 모델 with TF-IDF
# 랜덤포레스트 모델 with Word2Vec